<a href="https://colab.research.google.com/github/AbhaySrivastava26/GenAi/blob/master/Rnns/RnnGruLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhaysrivastavaa","key":"8d0af88146a1a4783e815088608ca8c0"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d shubhammaindola/harry-potter-books


Dataset URL: https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books
License(s): CC0-1.0
  0% 0.00/2.28M [00:00<?, ?B/s]
100% 2.28M/2.28M [00:00<00:00, 869MB/s]


In [12]:
import pandas as pd

df = pd.read_csv("Harry_Potter_Books.csv")  # adjust filename if different
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Harry_Potter_Books.csv'

In [4]:
!unzip -o harry-potter-books.zip
import os

# Folder where the files are unzipped (current directory)
files = [f for f in os.listdir() if f.endswith(".txt")]

# Read each book into a dictionary
books = {}
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        books[file] = f.read()

# Check
for title in books:
    print(title, "→", len(books[title]), "characters")



Archive:  harry-potter-books.zip
  inflating: 01 Harry Potter and the Sorcerers Stone.txt  
  inflating: 02 Harry Potter and the Chamber of Secrets.txt  
  inflating: 03 Harry Potter and the Prisoner of Azkaban.txt  
  inflating: 04 Harry Potter and the Goblet of Fire.txt  
  inflating: 05 Harry Potter and the Order of the Phoenix.txt  
  inflating: 06 Harry Potter and the Half-Blood Prince.txt  
  inflating: 07 Harry Potter and the Deathly Hallows.txt  
05 Harry Potter and the Order of the Phoenix.txt → 1499410 characters
03 Harry Potter and the Prisoner of Azkaban.txt → 625735 characters
02 Harry Potter and the Chamber of Secrets.txt → 492297 characters
04 Harry Potter and the Goblet of Fire.txt → 1100515 characters
06 Harry Potter and the Half-Blood Prince.txt → 987992 characters
07 Harry Potter and the Deathly Hallows.txt → 1140011 characters
01 Harry Potter and the Sorcerers Stone.txt → 439478 characters


In [ ]:
file_path = "01 Harry Potter and the Sorcerers Stone.txt"


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,Embedding,Dense
import numpy as np
#load and preprocess text data
def load_data(file_path):
    with open(file_path,'r',encoding='utf-8') as f:
        text=f.read()
    return text
file_path = "01 Harry Potter and the Sorcerers Stone.txt"

text=load_data(file_path).lower()
# print(text)
#Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
tokenizer=Tokenizer(oov_token='')#outofvocaublary
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(total_words)
# Convert text to sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0] # converts the input text into a list of numbers based on the word index
seq_length = 50  # Each input sequence contains 50 words

# First seq_length tokens (input): Used for training the model.
# Last token (target): Used as the label the model tries to predict.
# so total of (50 + 1) in one input_sequence index

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

# Pad sequences and split inputs/targets
# after this X will have inputs and y will have label for those inputs

input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# One-hot encode the labels , note- there are other ways for
# encoding like pre-trained word2vec encoding and so on

y = tf.keras.utils.to_categorical(y, num_classes=total_words)
# Build the Simple RNN model
model = Sequential([
    Embedding(input_dim=total_words, output_dim=64, input_length=seq_length),  # Word embeddings
    SimpleRNN(256, return_sequences=False),  # RNN Layer
    Dense(256, activation='relu'),  # Fully Connected Layer
    Dense(total_words, activation='softmax')  # Output Layer
])

# 256 in RNN - The number of hidden units (size of the hidden state vector)
# return_sequences=False  - The RNN will only return the final hidden state after processing the entire sequence

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=30, batch_size=128)


def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        tokenized_input = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_input = pad_sequences([tokenized_input], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(tokenized_input, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, "")

        seed_text += " " + predicted_word
    return seed_text

# Generate text using the trained model
print(generate_text(" to trust hagrid with something"))


6663


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.0421 - loss: 6.9234
Epoch 2/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0736 - loss: 6.1860
Epoch 3/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1030 - loss: 5.7495
Epoch 4/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1234 - loss: 5.4085
Epoch 5/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1362 - loss: 5.1307
Epoch 6/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1501 - loss: 4.8938
Epoch 7/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.1606 - loss: 4.6605
Epoch 8/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1670 - loss: 4.4842
Epoch 9/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1803 - loss: 4.2301
Epoch 10/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1970 - loss: 4.0023
Epoch 11/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.2176 - loss: 3.8080
Epoch 12/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 10s

In [6]:
def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        tokenized_input = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_input = pad_sequences([tokenized_input], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(tokenized_input, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, "")

        seed_text += " " + predicted_word
    return seed_text

# Generate text using the trained model
print(generate_text(" to trust hagrid with something"))

 to trust hagrid with something getting away with a nice toffee don’t you ” he cast sure they were saying “the potters that’s a real password to get the norwegian heads with the teachers mr weasley had hit him off the way down the white haired friend he’d never hoped that voldemort’s ever swallowed loudest


In [7]:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# Function to load dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

# Load Harry Potter book text
file_path = "01 Harry Potter and the Sorcerers Stone.txt"
text = load_data(file_path).lower()

# Tokenize the text
tokenizer = Tokenizer(oov_token='')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]
seq_length = 50  # Each input sequence will have 50 words

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

# Pad sequences and split into inputs (X) and labels (y)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)  # One-hot encode labels

# LSTM Model
model = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=seq_length),
    LSTM(256, return_sequences=True),  # First LSTM layer
    LSTM(256),  # Second LSTM layer
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=20, batch_size=128)

# Function to Generate Text
def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs) / temperature  # Adjust randomness
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))
        predicted_index = np.random.choice(range(len(predicted_probs)), p=predicted_probs)

        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word

    return seed_text

# Generate text
print(generate_text("harry looked at", next_words=50, temperature=0.7))

Epoch 1/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.0436 - loss: 7.0792
Epoch 2/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - accuracy: 0.0495 - loss: 6.4425
Epoch 3/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.0662 - loss: 6.1436
Epoch 4/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0916 - loss: 5.8753
Epoch 5/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.1055 - loss: 5.6385
Epoch 6/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.1156 - loss: 5.4305
Epoch 7/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.1260 - loss: 5.2362
Epoch 8/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.1253 - loss: 5.2920
Epoch 9/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.1390 - loss: 5.0476
Epoch 10/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.1479 - loss: 4.8432
Epoch 11/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1586 - loss: 4.6494
Epoch 12/20
633/633 ━━━━━━━━━━

In [10]:
import tensorflow as tf


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np

# Function to load dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

# Load Harry Potter book text
file_path = "01 Harry Potter and the Sorcerers Stone.txt"
text = load_data(file_path).lower()

# Tokenize the text
tokenizer = Tokenizer(oov_token='')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]
seq_length = 50  # Each input sequence will have 50 words

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

# Pad sequences and split into inputs (X) and labels (y)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)  # One-hot encode labels

# GRU Model
model = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=seq_length),
    GRU(256, return_sequences=True),  # First GRU layer
    GRU(256),  # Second GRU layer
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=20, batch_size=128)

# Function to Generate Text
def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs) / temperature  # Adjust randomness
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))
        predicted_index = np.random.choice(range(len(predicted_probs)), p=predicted_probs)

        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word

    return seed_text

# Generate text
print(generate_text("harry looked at", next_words=50, temperature=0.7))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0415 - loss: 7.1911
Epoch 2/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.0451 - loss: 6.7642
Epoch 3/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.0451 - loss: 6.6126
Epoch 4/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.0629 - loss: 6.3527
Epoch 5/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0876 - loss: 5.9757
Epoch 6/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.1192 - loss: 5.5126
Epoch 7/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.1413 - loss: 5.0336
Epoch 8/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.1659 - loss: 4.6151
Epoch 9/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.1976 - loss: 4.2624
Epoch 10/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.2387 - loss: 3.9043
Epoch 11/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.2785 - loss: 3.6046
Epoch 12/20
633/633 ━━━━━━━━━━